# Argumentation Avancee

Ce notebook couvre ADF, frameworks ponderes, sociaux et probabilistes.

**Pre-requis**: Executez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement.

**Navigation**: [Precedent](Tweety-6-Structured-Argumentation.ipynb) | [Suivant](Tweety-1-Setup.ipynb)


In [ ]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration des outils externes (standalone) ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str: 
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    # Also check if it's a directory (for Tweety compatibility)
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# Auto-detection Clingo (binaire)
# IMPORTANT: TweetyProject ClingoSolver expects a DIRECTORY path, not the full executable path.
# It appends "/clingo" (Unix) or "/clingo.exe" (Windows) internally when invoking the solver.
clingo_paths = [
    shutil.which("clingo"),
    shutil.which("clingo.exe"),
    pathlib.Path("ext_tools/clingo/clingo.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/clingo/clingo.exe"),
    pathlib.Path("../ext_tools/clingo/clingo.exe"),
]
for cp in clingo_paths:
    if cp:
        if isinstance(cp, str):
            # For system PATH clingo, use the directory containing it
            cp_path = pathlib.Path(cp)
            EXTERNAL_TOOLS["CLINGO"] = str(cp_path.parent.resolve())
            break
        elif cp.exists():
            # For local clingo.exe, store the DIRECTORY path (parent)
            EXTERNAL_TOOLS["CLINGO"] = str(cp.parent.resolve())
            break

# Auto-detection SPASS
spass_paths = [
    shutil.which("SPASS"),
    shutil.which("SPASS.exe"),
    pathlib.Path("ext_tools/spass/SPASS.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/spass/SPASS.exe"),
    pathlib.Path("../ext_tools/spass/SPASS.exe"),
]
for sp in spass_paths:
    if sp:
        if isinstance(sp, str):
            EXTERNAL_TOOLS["SPASS"] = sp
            break
        elif sp.exists():
            EXTERNAL_TOOLS["SPASS"] = str(sp.resolve())
            break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = pathlib.Path("jdk-17-portable")
    if not jdk_portable.exists():
        jdk_portable = pathlib.Path("Argument_Analysis/jdk-17-portable")
    
    if jdk_portable.exists():
        zulu_dirs = list(jdk_portable.glob("zulu*"))
        if zulu_dirs:
            java_home = zulu_dirs[0]
            os.environ["JAVA_HOME"] = str(java_home.resolve())
            print(f"JDK portable trouve: {java_home.name}")
    
    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
        print("JVM non disponible.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("Argument_Analysis/libs")
        
        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

if jvm_ready:
    print("JVM prete pour Tweety.")
    tools = [t for t, p in EXTERNAL_TOOLS.items() if p]
    if tools:
        print(f"Outils externes: {', '.join(tools)}")


## Partie 5 : Argumentation Avancée et Analyse
<a id="partie5"></a>

Cette section explore des extensions plus complexes et spécifiques des cadres d'argumentation, ainsi que des méthodes d'analyse dédiées comme le classement ou la prise en compte des probabilités. Ces formalismes permettent de modéliser des scénarios de raisonnement plus riches.

### 5.1 Abstract Dialectical Frameworks (ADF)
<a id="5.1"></a>

Les ADF [Brewka et al., 2013] généralisent les cadres de Dung. Au lieu d'une simple relation d'attaque, chaque argument (ou "statement") se voit associer une **condition d'acceptation** ($\phi_s$), qui est une formule propositionnelle définie sur les arguments parents (ceux qui ont un lien vers $s$). Le statut d'un argument dépend de la satisfaction de sa condition d'acceptation en fonction du statut de ses parents.

* **Structure :** Un ADF est un triplet $(S, L, C)$ où $S$ est l'ensemble des "statements" (arguments), $L \subseteq S \times S$ sont les liens, et $C = \{\phi_s\}_{s \in S}$ est l'ensemble des conditions d'acceptation.
* **Conditions d'Acceptation ($\phi_s$) :** Une fonction booléenne $Val(par(s)) \to \{\mathbf{t}, \mathbf{f}\}$ où $par(s)$ sont les parents de $s$. Souvent exprimée comme une formule propositionnelle (ex: `a & (~b | c)`).
* **Sémantiques :** Généralisent celles de Dung (admissible, complète, fondée, préférée, stable, modèle/2-valué). Le raisonnement est souvent coûteux (plus complexe que pour les AAF) et typiquement réduit à des problèmes SAT ou QBF.
* **Tweety :** `AbstractDialecticalFramework`, `Argument`, `AcceptanceCondition`, `KppADFFormatParser` (pour lire un format texte spécifique). Les raisonneurs (`arg.adf.reasoner.*`) utilisent souvent des solveurs SAT natifs (`arg.adf.sat.solver.*` - ex: `NativeMinisatSolver`) qui nécessitent les bibliothèques `.dll` ou `.so` téléchargées précédemment (voir note Cellule 10). Si ces bibliothèques natives ne sont pas trouvées/utilisables par la JVM (problème `java.library.path`), le raisonnement ADF risque d'échouer.

In [ ]:
# --- 5.1 Abstract Dialectical Frameworks (ADF) ---
print("\n--- 5.1 Abstract Dialectical Frameworks (ADF) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple ADF...")
    print("⚠️ NOTE IMPORTANTE : Le raisonnement ADF dans Tweety repose fortement sur des")
    print("         solveurs SAT natifs (Minisat, Lingeling, Picosat) qui implémentent")
    print("         l'interface 'IncrementalSatSolver'.")

    adf_imports_ok = False
    native_solver_load_failed = False
    try:
        # Imports
        import jpype; from jpype.types import *; import pathlib
        from java.io import File; from java.util import Collection

        from org.tweetyproject.arg.adf.syntax.adf import AbstractDialecticalFramework
        from org.tweetyproject.arg.adf.syntax import Argument as AdfArgument
        from org.tweetyproject.arg.adf.syntax.acc import AcceptanceCondition
        from org.tweetyproject.arg.adf.io import KppADFFormatParser
        from org.tweetyproject.arg.adf.semantics.link import SatLinkStrategy, LinkStrategy
        from org.tweetyproject.arg.adf.semantics.interpretation import Interpretation
        from org.tweetyproject.arg.adf.reasoner import AdmissibleReasoner, CompleteReasoner, GroundReasoner, PreferredReasoner, StableReasoner, ModelReasoner
        from org.tweetyproject.logics.pl.sat import Sat4jSolver # Fallback
        # Interface requise par SatLinkStrategy
        from org.tweetyproject.arg.adf.sat import IncrementalSatSolver

        print("✔️ Imports ADF de base réussis.")
        adf_imports_ok = True

        # Tenter de charger un solveur natif
        adf_solver = None
        try:
             from org.tweetyproject.arg.adf.sat.solver import NativeMinisatSolver
             adf_solver = NativeMinisatSolver()
             print("ℹ️ Utilisation tentée de NativeMinisatSolver.")
        except Exception as e_native_init:
             native_solver_load_failed = True
             print(f"⚠️ Échec chargement/init NativeMinisatSolver (attendu sans java.library.path): {e_native_init}")
             print("   Utilisation du fallback Sat4jSolver...")
             adf_solver = Sat4jSolver()

        # --- Tentative d'initialisation du Parser ADF ---
        print("\nTentative de création de SatLinkStrategy...")
        link_strategy = None
        try:
            # SatLinkStrategy attend un IncrementalSatSolver
            # Vérifions si notre solveur l'est (le natif oui, Sat4j non)
            IncrementalSatSolver_class = jpype.JClass("org.tweetyproject.arg.adf.sat.IncrementalSatSolver")

            # *** CORRECTION: Utiliser isinstance() Python ***
            if isinstance(adf_solver, IncrementalSatSolver_class):
                 print(f"   Le solveur ({adf_solver.getClass().getSimpleName()}) est un IncrementalSatSolver.")
                 link_strategy = SatLinkStrategy(adf_solver) # Devrait fonctionner si natif chargé
                 print("✔️ SatLinkStrategy créée avec succès.")
            else:
                 print(f"   ❌ Le solveur fallback ({adf_solver.getClass().getSimpleName()}) N'EST PAS un IncrementalSatSolver.")
                 print("      Impossible de créer SatLinkStrategy requis par KppADFFormatParser.")
                 # On ne tente même pas de l'appeler car on sait que ça échouera avec TypeError

        except jpype.JException as e_link_java:
             print(f"❌ Erreur Java lors de la création de SatLinkStrategy: {e_link_java.message()}")
        except Exception as e_link_py:
              print(f"❌ Erreur Python lors de la création de SatLinkStrategy: {e_link_py}")


        # --- Conclusion ADF ---
        if link_strategy is None:
             print("\n❌ CONCLUSION ADF : Impossible de procéder.")
             print("   Le raisonnement ADF nécessite un solveur SAT incrémental natif.")
             if native_solver_load_failed:
                 print("   Celui-ci n'a pas pu être chargé (UnsatisfiedLinkError probable),")
             print("   et le fallback Sat4jSolver n'est pas compatible avec SatLinkStrategy.")
             print("   Pour faire fonctionner cette section, il faudrait configurer")
             print("   le java.library.path de la JVM pour inclure le dossier 'libs/native',")
             print("   ce qui est complexe à faire proprement avec JPype.")

        # --- Code commenté (ne peut pas fonctionner sans link_strategy) ---
        # print("\nLe parsing et le raisonnement ADF sont commentés car non fonctionnels dans cet état.")
        '''
        # --- Parsing ADF depuis Fichier ---
        adf_filename = "adf_example.txt"
        adf_filepath = pathlib.Path("resources") / adf_filename
        adf_framework = None

        if not adf_filepath.is_file():
            print(f"\\n❌ ERREUR: Fichier ADF requis '{adf_filepath}' non trouvé !")
        elif link_strategy:
            print(f"\\nChargement de l'ADF depuis: {adf_filepath}")
            try:
                adf_parser = KppADFFormatParser(link_strategy, True)
                adf_framework = adf_parser.parse(File(str(adf_filepath)))
                print("✔️ ADF chargé avec succès depuis le fichier.")
                # ... (affichage) ...
            except Exception as e_parse_adf: print(f"❌ Erreur parsing ADF: {e_parse_adf}")

        # --- Raisonnement ADF ---
        if adf_framework is not None:
            print("\\n--- Calcul des Sémantiques ADF ---")
            # ... (raisonneurs) ...
        else:
            print("\\n❌ Aucun ADF chargé, raisonnement sauté.")
        '''

    # ... (Blocs except globaux inchangés) ...
    except ImportError as e: print(f"❌ Erreur d'import pour ADF : {e}. Vérifiez le JAR 'arg.adf'.")
    except jpype.JException as e_java: print(f"❌ Erreur Java générale: {e_java.message()}")
    except Exception as e_gen: print(f"❌ Erreur Python inattendue: {e_gen}"); import traceback; traceback.print_exc()

### 5.2 Frameworks Bipolaires (EAF, PEAF, Evidential, Necessity)
<a id="5.2"></a>

Les cadres d'argumentation bipolaires étendent explicitement les AAFs de Dung en introduisant une relation de **support** distincte de l'attaque. Différentes propositions existent pour définir la nature exacte de ce support et son interaction avec l'attaque. Tweety implémente plusieurs de ces formalismes :

* **EAF (Evidential/Extended AF)** (`org.tweetyproject.arg.bipolar.syntax.EAFTheory`): Introduit un support ensembliste simple (un ensemble d'arguments supporte un autre ensemble). Le raisonnement se fait souvent par traduction vers un AAF standard.
* **PEAF (Probabilistic EAF)** (`org.tweetyproject.arg.bipolar.syntax.PEAFTheory`): Ajoute une probabilité à chaque relation de support, permettant de modéliser l'incertitude du support. Utilisé pour l'analyse de justification probabiliste.
* **Evidential AF** (`org.tweetyproject.arg.bipolar.syntax.EvidentialArgumentationFramework`): Framework avec support ensembliste et la notion d'arguments "prima facie" (acceptés par défaut). Possède ses propres sémantiques (SelfSupporting, ConflictFree, Admissible...).
* **Necessity AF** (`org.tweetyproject.arg.bipolar.syntax.NecessityArgumentationFramework`): Support ensembliste interprété comme une "nécessité" (tous les arguments supportants sont requis). Possède également ses propres sémantiques.

Les exemples suivants illustreront l'Evidential AF et le Necessity AF.

In [ ]:
# --- 5.2 Frameworks Bipolaires ---
print("\n--- 5.2 Frameworks Bipolaires ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution des exemples Bipolaires...")
    bipolar_imports_ok = False
    try:
        # Imports
        import jpype
        from jpype.types import *
        from java.util import HashSet, Collection, Set as JavaSet

        # Syntaxe Dung de base (Argument) et Bipolaire
        from org.tweetyproject.arg.dung.syntax import Argument
        from org.tweetyproject.arg.bipolar.syntax import (
             EvidentialArgumentationFramework, NecessityArgumentationFramework,
             BArgument, ArgumentSet,
             Attack, Support, # Interfaces/Classes de base
             BinaryAttack, BinarySupport, # Pour Necessity
             SetAttack, SetSupport # Pour Evidential et Necessity
             )
        # Raisonneurs Evidential
        from org.tweetyproject.arg.bipolar.reasoner.evidential import (
            SelfSupportingReasoner, ConflictFreeReasoner as EvidentialConflictFreeReasoner,
            AdmissibleReasoner as EvidentialAdmissibleReasoner, GroundedReasoner as EvidentialGroundedReasoner,
            CompleteReasoner as EvidentialCompleteReasoner, PreferredReasoner as EvidentialPreferredReasoner,
            StableReasoner as EvidentialStableReasoner
            )
        # Raisonneurs Necessity
        from org.tweetyproject.arg.bipolar.reasoner.necessity import (
            AdmissibleReasoner as NecessityAdmissibleReasoner, GroundedReasoner as NecessityGroundedReasoner,
            CompleteReasoner as NecessityCompleteReasoner, PreferredReasoner as NecessityPreferredReasoner,
            StableReasoner as NecessityStableReasoner
            )

        print("✔️ Imports pour Frameworks Bipolaires réussis.")
        bipolar_imports_ok = True

        # --- Exemple Evidential Argumentation ---
        if bipolar_imports_ok:
            print("\n--- Exemple Evidential AF ---")
            et = EvidentialArgumentationFramework()
            args_ev = {name: BArgument(name) for name in "abcdef"}
            for arg in args_ev.values(): et.add(arg) # add(Argument) est ok

            # Créer les objets Attack/Support
            attack_b_a = SetAttack(args_ev['b'], args_ev['a'])
            attack_b_c = SetAttack(args_ev['b'], args_ev['c'])
            attack_c_b = SetAttack(args_ev['c'], args_ev['b'])
            attack_c_d = SetAttack(args_ev['c'], args_ev['d'])
            attack_d_f = SetAttack(args_ev['d'], args_ev['f'])
            attack_f_f = SetAttack(args_ev['f'], args_ev['f'])
            support_d_e = SetSupport(args_ev['d'], args_ev['e'])

            # *** CORRECTION: Utiliser getAttacks().add() et getSupports().add() ***
            print("   Ajout des attaques et supports (Evidential)...")
            attacks_et = et.getAttacks() # Récupérer la Collection<Attack>
            supports_et = et.getSupports() # Récupérer la Collection<Support>
            attacks_et.add(attack_b_a)
            attacks_et.add(attack_b_c)
            attacks_et.add(attack_c_b)
            attacks_et.add(attack_c_d)
            attacks_et.add(attack_d_f)
            attacks_et.add(attack_f_f)
            supports_et.add(support_d_e)
            print("   ✔️ Relations ajoutées.")

            # Arguments Prima Facie (inchangé)
            et.addPrimaFacie(args_ev['b'])
            et.addPrimaFacie(args_ev['c'])
            et.addPrimaFacie(args_ev['d'])
            et.addPrimaFacie(args_ev['f'])

            print("\nFramework Evidential créé:")
            print(str(et.prettyPrint()))

            # Raisonnement Evidential (inchangé)
            print("\nCalcul des extensions Evidential:")
            try:
                print(f" - Self-Supporting: {SelfSupportingReasoner().getModels(et)}")
                print(f" - Conflict-Free: {EvidentialConflictFreeReasoner().getModels(et)}")
                print(f" - Admissible: {EvidentialAdmissibleReasoner().getModels(et)}")
                print(f" - Grounded: {EvidentialGroundedReasoner().getModels(et)}")
                print(f" - Complete: {EvidentialCompleteReasoner().getModels(et)}")
                print(f" - Preferred: {EvidentialPreferredReasoner().getModels(et)}")
                print(f" - Stable: {EvidentialStableReasoner().getModels(et)}")
            except Exception as e_ev_reason:
                print(f"❌ Erreur raisonnement Evidential: {e_ev_reason}")


            # --- Exemple Necessity Argumentation ---
            print("\n\n--- Exemple Necessity AF ---")
            nt = NecessityArgumentationFramework()
            args_nec = {name: BArgument(name) for name in "abcde"}
            for arg in args_nec.values(): nt.add(arg) # add(Argument) ok

            # Créer les objets Attack/Support
            attack_b_a_n = BinaryAttack(args_nec['b'], args_nec['a'])
            attack_e_a_n = BinaryAttack(args_nec['e'], args_nec['a'])
            attack_c_d_n = BinaryAttack(args_nec['c'], args_nec['d'])
            support_a_c_n = BinarySupport(args_nec['a'], args_nec['c'])
            support_b_b_n = BinarySupport(args_nec['b'], args_nec['b'])
            # Pour SetSupport
            supportants_s3 = ArgumentSet()
            supportants_s3.add(args_nec['b'])
            supportants_s3.add(args_nec['d'])
            supportes_s3 = HashSet()
            supportes_s3.add(args_nec['e'])
            support_set_n = SetSupport(supportants_s3, JObject(supportes_s3, JavaSet))

            # *** CORRECTION: Utiliser getAttacks().add() et getSupports().add() ***
            print("   Ajout des attaques et supports (Necessity)...")
            attacks_nt = nt.getAttacks()
            supports_nt = nt.getSupports()
            attacks_nt.add(attack_b_a_n)
            attacks_nt.add(attack_e_a_n)
            attacks_nt.add(attack_c_d_n)
            supports_nt.add(support_a_c_n)
            supports_nt.add(support_b_b_n)
            supports_nt.add(support_set_n)
            print("   ✔️ Relations ajoutées.")


            print("\nFramework Necessity créé:")
            print(str(nt.prettyPrint()))

             # Raisonnement Necessity (inchangé)
            print("\nCalcul des extensions Necessity:")
            try:
                print(f" - Admissible: {NecessityAdmissibleReasoner().getModels(nt)}")
                print(f" - Grounded: {NecessityGroundedReasoner().getModels(nt)}")
                print(f" - Complete: {NecessityCompleteReasoner().getModels(nt)}")
                print(f" - Preferred: {NecessityPreferredReasoner().getModels(nt)}")
                print(f" - Stable: {NecessityStableReasoner().getModels(nt)}")
            except Exception as e_nec_reason:
                print(f"❌ Erreur raisonnement Necessity: {e_nec_reason}")


        else:
             print("❌ Imports Bipolaires échoués. Impossible de continuer l'exemple.")

    # Gestion globale (inchangée)
    except ImportError as e: print(f"❌ Erreur d'import Bipolaires: {e}.")
    except jpype.JException as e_java: print(f"❌ Erreur Java générale Bipolaires: {e_java.message()}"); print(e_java.stacktrace())
    except Exception as e_gen: print(f"❌ Erreur Python inattendue Bipolaires: {e_gen}"); import traceback; traceback.print_exc()

### 5.3 Frameworks Pondérés (WAF)
<a id="5.3"></a>

Les Cadres d'Argumentation Pondérés (Weighted Argumentation Frameworks - WAF) étendent les cadres de Dung en associant un **poids** à chaque attaque, représentant sa force ou son coût. L'évaluation de l'acceptabilité des arguments prend alors en compte ces poids, souvent à travers des **seuils** ou des **agrégations**.

* **Structure :** Un WAF est un triplet $(Args, Atts, w)$ où $(Args, Atts)$ est un AAF de Dung et $w: Atts \\to \\mathcal{W}$ est une fonction qui assigne un poids de l'ensemble $\\mathcal{W}$ à chaque attaque.
* **Semi-anneaux (`Semiring`) :** Tweety utilise la structure algébrique des semi-anneaux pour définir comment les poids sont interprétés et agrégés. Exemples courants :
    * `WeightedSemiring`: Poids numériques sommés (souvent interprétés comme des coûts).
    * `FuzzySemiring`: Poids dans $[0, 1]$ interprétés comme des degrés de vérité/force (logique floue).
    * `ProbabilisticSemiring`: Poids dans $[0, 1]$ interprétés comme des probabilités.
    * `BottleneckSemiring`: L'agrégation prend le "maillon faible" (max ou min selon l'interprétation).
    * `BooleanSemiring`: Poids booléens, équivalent à un AAF standard.
* **`WeightedArgumentationFramework<S>`**: Classe Tweety paramétrée par le type de semi-anneau `S`.
* **Raisonnement (`SimpleWeighted...Reasoner`)**: Les raisonneurs pour WAF généralisent les sémantiques standards (admissible, complète, etc.). Ils prennent souvent des **seuils** (`alpha`, `gamma`) en paramètres pour déterminer quand un argument est considéré comme "suffisamment défendu" ou quand une attaque est "suffisamment forte" pour être prise en compte.

L'exemple suivant implémente `WeightedReasonerExample.java` en utilisant `WeightedSemiring`.

In [ ]:
# --- 5.3 Frameworks Pondérés (WAF) ---
print("\n--- 5.3 Frameworks Pondérés (WAF) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple WAF...")
    waf_imports_ok = False
    try:
        # Imports
        import jpype
        from jpype.types import *

        # WAF et composants Dung
        from org.tweetyproject.arg.weighted.syntax import WeightedArgumentationFramework
        from org.tweetyproject.arg.dung.syntax import Argument, Attack, DungTheory
        # Semi-anneau spécifique (autres possibles: FuzzySemiring, ProbabilisticSemiring, etc.)
        from org.tweetyproject.math.algebra import WeightedSemiring
        # Raisonneurs pondérés
        from org.tweetyproject.arg.weighted.reasoner import (
             SimpleWeightedConflictFreeReasoner, SimpleWeightedAdmissibleReasoner,
             SimpleWeightedCompleteReasoner, SimpleWeightedPreferredReasoner,
             SimpleWeightedStableReasoner, SimpleWeightedGroundedReasoner
        )
        # Raisonneurs Dung standards pour comparaison
        from org.tweetyproject.arg.dung.reasoner import (
            SimpleAdmissibleReasoner, SimpleCompleteReasoner, SimplePreferredReasoner,
            SimpleStableReasoner, SimpleGroundedReasoner
        )
        from org.tweetyproject.arg.dung.semantics import Extension # Pour type hinting
        from java.util import Collection

        print("✔️ Imports WAF et dépendances réussis.")
        waf_imports_ok = True

        # --- Création du WAF ---
        if waf_imports_ok:
            # Utiliser WeightedSemiring (poids numériques additifs/coûts)
            # On peut spécifier une valeur max (comme 20.0 dans l'exemple Java) ou non
            semiring = WeightedSemiring() # Par défaut, pas de max explicitement défini ici

            # WeightedArgumentationFramework<S extends Semiring>
            waf = WeightedArgumentationFramework(semiring)

            # Arguments
            a = Argument("a"); b = Argument("b"); c = Argument("c"); d = Argument("d"); e_arg = Argument("e") # Renommé e -> e_arg
            waf.add(a); waf.add(b); waf.add(c); waf.add(d); waf.add(e_arg)

            # Attaques pondérées: add(Attack, Double weight)
            waf.add(Attack(a, b), 7.0)
            waf.add(Attack(c, b), 8.0)
            waf.add(Attack(d, c), 8.0)
            waf.add(Attack(c, d), 9.0)
            waf.add(Attack(d, e_arg), 5.0)
            waf.add(Attack(e_arg, e_arg), 6.0) # Auto-attaque sur e

            print("\nFramework Pondéré (WAF) créé:")
            print(str(waf)) # Devrait montrer les poids

            # --- Raisonnement Pondéré ---
            print("\n--- Raisonnement Pondéré ---")

            # Raisonneur Conflict-Free Pondéré
            print("\n* Conflict-Free Pondéré:")
            try:
                weighted_cf_reasoner = SimpleWeightedConflictFreeReasoner()
                # getModels(WAF, threshold) - Le seuil n'est généralement pas pertinent pour CF seul
                # L'implémentation Java semble l'ignorer, mettons 0.0
                cf_sets_0 = weighted_cf_reasoner.getModels(waf, 0.0)
                cf_sets_15 = weighted_cf_reasoner.getModels(waf, 15.0) # Test avec seuil > max poids
                print(f"  - Seuil 0.0 : ({cf_sets_0.size()}) {cf_sets_0}")
                print(f"  - Seuil 15.0: ({cf_sets_15.size()}) {cf_sets_15}")
            except Exception as e_cf: print(f"   ❌ Erreur Weighted ConflictFree: {e_cf}")


            # Raisonneur Admissible Pondéré vs Standard
            print("\n* Admissible Pondéré (alpha, gamma) vs Standard:")
            try:
                adm_reasoner_std = SimpleAdmissibleReasoner()
                weighted_adm_reasoner = SimpleWeightedAdmissibleReasoner()
                dung_theory_std = DungTheory(waf) # Version non pondérée pour comparaison

                adm_sets_std = adm_reasoner_std.getModels(dung_theory_std)
                print(f"  - Standard    : ({adm_sets_std.size()}) {adm_sets_std}")

                # getModels(WAF, alpha, gamma)
                adm_sets_0_0 = weighted_adm_reasoner.getModels(waf, 0.0, 0.0)
                print(f"  - Pondéré (α=0, γ=0) : ({adm_sets_0_0.size()}) {adm_sets_0_0}")
                adm_sets_15_0 = weighted_adm_reasoner.getModels(waf, 15.0, 0.0)
                print(f"  - Pondéré (α=15, γ=0): ({adm_sets_15_0.size()}) {adm_sets_15_0}")
                adm_sets_11_1 = weighted_adm_reasoner.getModels(waf, 11.0, 1.0)
                print(f"  - Pondéré (α=11, γ=1): ({adm_sets_11_1.size()}) {adm_sets_11_1}")

            except Exception as e_adm: print(f"   ❌ Erreur Weighted Admissible: {e_adm}")


            # Raisonneur Complet Pondéré vs Standard
            print("\n* Complet Pondéré (alpha, gamma) vs Standard:")
            try:
                comp_reasoner_std = SimpleCompleteReasoner()
                weighted_comp_reasoner = SimpleWeightedCompleteReasoner()
                comp_sets_std = comp_reasoner_std.getModels(dung_theory_std)
                print(f"  - Standard    : ({comp_sets_std.size()}) {comp_sets_std}")

                comp_sets_0_0 = weighted_comp_reasoner.getModels(waf, 0.0, 0.0)
                print(f"  - Pondéré (α=0, γ=0) : ({comp_sets_0_0.size()}) {comp_sets_0_0}")
                comp_sets_0_1 = weighted_comp_reasoner.getModels(waf, 0.0, 1.0)
                print(f"  - Pondéré (α=0, γ=1) : ({comp_sets_0_1.size()}) {comp_sets_0_1}")
                comp_sets_11_1 = weighted_comp_reasoner.getModels(waf, 11.0, 1.0)
                print(f"  - Pondéré (α=11, γ=1): ({comp_sets_11_1.size()}) {comp_sets_11_1}")
            except Exception as e_comp: print(f"   ❌ Erreur Weighted Complete: {e_comp}")


            # Raisonneur Préféré Pondéré vs Standard
            print("\n* Préféré Pondéré (alpha, gamma) vs Standard:")
            try:
                pref_reasoner_std = SimplePreferredReasoner()
                weighted_pref_reasoner = SimpleWeightedPreferredReasoner()
                pref_sets_std = pref_reasoner_std.getModels(dung_theory_std)
                print(f"  - Standard    : ({pref_sets_std.size()}) {pref_sets_std}")

                pref_sets_0_0 = weighted_pref_reasoner.getModels(waf, 0.0, 0.0)
                print(f"  - Pondéré (α=0, γ=0) : ({pref_sets_0_0.size()}) {pref_sets_0_0}")
                pref_sets_0_1 = weighted_pref_reasoner.getModels(waf, 0.0, 1.0)
                print(f"  - Pondéré (α=0, γ=1) : ({pref_sets_0_1.size()}) {pref_sets_0_1}")
            except Exception as e_pref: print(f"   ❌ Erreur Weighted Preferred: {e_pref}")


            # Raisonneur Stable Pondéré vs Standard
            print("\n* Stable Pondéré (alpha, gamma) vs Standard:")
            try:
                stab_reasoner_std = SimpleStableReasoner()
                weighted_stab_reasoner = SimpleWeightedStableReasoner()
                stab_sets_std = stab_reasoner_std.getModels(dung_theory_std)
                print(f"  - Standard    : ({stab_sets_std.size()}) {stab_sets_std}")

                stab_sets_0_0 = weighted_stab_reasoner.getModels(waf, 0.0, 0.0)
                print(f"  - Pondéré (α=0, γ=0) : ({stab_sets_0_0.size()}) {stab_sets_0_0}")
                stab_sets_0_1 = weighted_stab_reasoner.getModels(waf, 0.0, 1.0)
                print(f"  - Pondéré (α=0, γ=1) : ({stab_sets_0_1.size()}) {stab_sets_0_1}")
            except Exception as e_stab: print(f"   ❌ Erreur Weighted Stable: {e_stab}")

            # Raisonneur Grounded Pondéré vs Standard
            print("\n* Grounded Pondéré (alpha, gamma) vs Standard:")
            try:
                gr_reasoner_std = SimpleGroundedReasoner()
                weighted_gr_reasoner = SimpleWeightedGroundedReasoner()
                gr_set_std = gr_reasoner_std.getModel(dung_theory_std) # Grounded est unique
                print(f"  - Standard    : {{{gr_set_std}}}") # Afficher comme set

                gr_set_0_0 = weighted_gr_reasoner.getModel(waf, 0.0, 0.0)
                print(f"  - Pondéré (α=0, γ=0) : {{{gr_set_0_0}}}")
            except Exception as e_gr: print(f"   ❌ Erreur Weighted Grounded: {e_gr}")


        else:
             print("❌ Imports WAF échoués. Impossible de continuer l'exemple.")

    # Gestion globale
    except ImportError as e:
        print(f"❌ Erreur d'import pour WAF : {e}. Vérifiez les JARs 'arg.weighted', 'arg.dung', 'math'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale WAF: {e_java.message()}")
        # print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue WAF: {e_gen}")
        import traceback; traceback.print_exc()

### 5.4 Frameworks Sociaux (SAF)
<a id="5.4"></a>

Les Cadres d'Argumentation Sociaux (SAF) [Leite, Martins, 2011] étendent les cadres de Dung en permettant d'associer des **votes** (positifs ou négatifs) aux arguments. Ces votes influencent la force ou l'acceptabilité finale des arguments, en plus des relations d'attaque.

* **Structure :** Un AAF de Dung + une fonction V qui assigne à chaque argument un support numérique (ex: nombre de votes positifs - nombre de votes négatifs).
* **Tweety :** `SocialAbstractArgumentationFramework` étend `DungTheory` et fournit les méthodes `voteUp(arg, count)` et `voteDown(arg, count)`.
* **Sémantique :** Souvent itérative, comme **ISS (Iterated Schema Semantics)** [Gabbay, Rodrigues], qui calcule un score pour chaque argument basé sur les scores de ses attaquants et sur les votes, jusqu'à convergence.
* **Raisonnement :** `IssReasoner` implémente cette sémantique. Il prend en paramètre une fonction de mise à jour (ex: `SimpleProductSemantics`) et un seuil de convergence.

In [ ]:
# --- 5.4 Frameworks Sociaux (SAF) ---
print("\n--- 5.4 Frameworks Sociaux (SAF) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple SAF...")
    saf_imports_ok = False
    try:
        # Imports SAF et Dung (Argument, Attack)
        import jpype
        from jpype.types import *
        # Utiliser JClass pour être sûr si les imports directs échouent
        try:
            from org.tweetyproject.arg.social.syntax import SocialAbstractArgumentationFramework
            from org.tweetyproject.arg.dung.syntax import Argument, Attack
            from org.tweetyproject.arg.social.reasoner import IssReasoner
            from org.tweetyproject.arg.social.semantics import SimpleProductSemantics
            print("✔️ Imports SAF directs réussis.")
            saf_imports_ok = True
        except ImportError:
            print("⚠️ Imports directs SAF échoués. Tentative avec JClass...")
            SocialAbstractArgumentationFramework = jpype.JClass("org.tweetyproject.arg.social.syntax.SocialAbstractArgumentationFramework")
            Argument = jpype.JClass("org.tweetyproject.arg.dung.syntax.Argument")
            Attack = jpype.JClass("org.tweetyproject.arg.dung.syntax.Attack")
            IssReasoner = jpype.JClass("org.tweetyproject.arg.social.reasoner.IssReasoner")
            SimpleProductSemantics = jpype.JClass("org.tweetyproject.arg.social.semantics.SimpleProductSemantics")
            print("✔️ Imports SAF via JClass réussis.")
            saf_imports_ok = True

        # Collection Java pour getAttacks()
        from java.util import Collection

        # --- Création du SAF ---
        if saf_imports_ok:
            saf = SocialAbstractArgumentationFramework()
            A = Argument("A"); B = Argument("B"); C = Argument("C"); D = Argument("D")
            saf.add(A); saf.add(B); saf.add(C); saf.add(D) # add(Argument) est non ambigu

            # Attaques: A->B, B<->C, C->D
            attack_ab = Attack(A, B)
            attack_bc = Attack(B, C)
            attack_cb = Attack(C, B)
            attack_cd = Attack(C, D)

            # *** NOUVELLE APPROCHE : Obtenir la collection et utiliser son add ***
            try:
                attacks_collection = saf.getAttacks() # Devrait retourner Collection<Attack>
                # Utiliser la méthode add de la Collection Java
                attacks_collection.add(attack_ab)
                attacks_collection.add(attack_bc)
                attacks_collection.add(attack_cb)
                attacks_collection.add(attack_cd)
                print("   ✔️ Attaques ajoutées avec succès (via getAttacks().add()).")

                # --- Ajout des Votes ---
                saf.voteUp(A, 3); saf.voteDown(A, 1) # A: +2 net
                saf.voteUp(B, 2)                     # B: +2 net
                saf.voteUp(C, 2); saf.voteDown(C, 5) # C: -3 net
                saf.voteUp(D, 2); saf.voteDown(D, 1) # D: +1 net

                print("\nFramework Social (SAF) créé:")
                print(str(saf))

                # --- Raisonnement ISS ---
                print("\nCalcul du modèle avec ISS (Iterated Schema Semantics)...")
                try:
                    iss_reasoner = IssReasoner(SimpleProductSemantics(0.01), 0.001)
                    iss_model = iss_reasoner.getModel(saf)
                    print("\nModèle ISS (scores d'acceptabilité):\n", str(iss_model))

                except jpype.JException as e_iss_java:
                     print(f"❌ Erreur Java lors du raisonnement ISS: {e_iss_java.message()}")
                     # print(e_iss_java.stacktrace())
                except Exception as e_iss_py:
                     print(f"❌ Erreur Python lors du raisonnement ISS: {e_iss_py}")

            except Exception as e_add_attack:
                 print(f"❌ Erreur lors de l'ajout d'attaques via getAttacks().add(): {e_add_attack}")
                 print("   Impossible de continuer avec cet exemple SAF.")


        else:
             print("❌ Imports SAF échoués. Impossible de continuer l'exemple.")


    # ... (Blocs except globaux inchangés) ...
    except ImportError as e: print(f"❌ Erreur d'import initiale pour SAF : {e}.")
    except jpype.JException as e_java: print(f"❌ Erreur Java générale: {e_java.message()}")
    except Exception as e_gen: print(f"❌ Erreur Python inattendue: {e_gen}"); import traceback; traceback.print_exc()

### 5.5 Set Argumentation Frameworks (SetAF)
<a id="5.5"></a>

Les Set Argumentation Frameworks (SetAF), introduits par Nielsen et Parsons (2006), généralisent les attaques des AAF de Dung. Au lieu qu'un argument unique attaque un autre argument, c'est un **ensemble** d'arguments qui attaque collectivement un argument.

* **Structure :** Un SetAF est une paire $(Args, Atts_{set})$ où $Args$ est l'ensemble d'arguments et $Atts_{set} \\subseteq (2^{Args} \\setminus \\emptyset) \\times Args$ est la relation d'attaque d'ensemble. Une attaque est de la forme $(X, a)$ où $X$ est un ensemble non vide d'arguments et $a$ est l'argument attaqué.
* **Sémantique :** Les sémantiques classiques (conflict-free, admissible, complète, préférée, fondée, stable) sont étendues aux SetAF. Par exemple, un ensemble $E$ est sans conflit si aucun sous-ensemble $X \\subseteq E$ n'attaque un argument $a \\in E$. Un argument $a$ est défendu par $E$ si pour chaque ensemble $X$ qui attaque $a$, il existe un $e \\in E$ tel qu'un sous-ensemble $Y \\subseteq E$ attaque un $x \\in X$.
* **Tweety :**
    * `SetAf` : Représente le framework.
    * `SetAttack` : Représente une attaque $(X, a)$. Prend un `Set<Argument>` et un `Argument`.
    * Raisonneurs (`org.tweetyproject.arg.setaf.reasoners.*`) : `SimpleGroundedSetAfReasoner`, `SimpleAdmissibleSetAfReasoner`, `SimplePreferredSetAfReasoner`.

L'exemple suivant implémente `SetAfTheoryTest.java`.

In [ ]:
# --- 5.5 Set Argumentation Frameworks (SetAF) ---
print("\n--- 5.5 Set Argumentation Frameworks (SetAF) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple SetAF...")
    setaf_imports_ok = False
    try:
        # Imports
        import jpype
        from jpype.types import *
        from java.util import HashSet, Set as JavaSet # Pour créer l'ensemble attaquant

        # Composants SetAF et Dung
        from org.tweetyproject.arg.setaf.syntax import SetAf, SetAttack
        from org.tweetyproject.arg.dung.syntax import Argument # Argument est commun
        # Raisonneurs SetAF
        from org.tweetyproject.arg.setaf.reasoners import SimpleGroundedSetAfReasoner, SimpleAdmissibleSetAfReasoner, SimplePreferredSetAfReasoner

        print("✔️ Imports SetAF et dépendances réussis.")
        setaf_imports_ok = True

        # --- Création du SetAF ---
        if setaf_imports_ok:
            set_af = SetAf()
            a = Argument("a"); b = Argument("b"); c = Argument("c"); d_arg = Argument("d") # Renommé d -> d_arg
            set_af.add(a); set_af.add(b); set_af.add(c); set_af.add(d_arg)

            # Créer les ensembles attaquants (HashSet Java)
            attacker_set1 = HashSet()
            attacker_set1.add(b)
            attacker_set1.add(d_arg)

            attacker_set2 = HashSet()
            attacker_set2.add(c)
            attacker_set2.add(a)

            # Ajouter les attaques d'ensemble: SetAttack(Set<Argument> attackers, Argument attacked)
            # Cast explicite du HashSet en java.util.Set pour le constructeur
            set_attack1 = SetAttack(JObject(attacker_set1, JavaSet), a)
            set_attack2 = SetAttack(JObject(attacker_set2, JavaSet), c)
            set_af.add(set_attack1)
            set_af.add(set_attack2)

            print("\nFramework SetAF créé:")
            # Utiliser toString() ou la représentation Python standard
            print(str(set_af))
            # Alternative: Afficher arguments et attaques séparément pour plus de clarté
            print(f"   Arguments: {set_af.getNodes()}")
            print(f"   Attaques Set: {set_af.getAttacks()}")


            # --- Raisonnement SetAF ---
            print("\n--- Raisonnement SetAF ---")
            try:
                gr_setaf_reasoner = SimpleGroundedSetAfReasoner()
                gr_setaf_extension = gr_setaf_reasoner.getModel(set_af)
                print(f"\n* Extension Fondée (Grounded): {gr_setaf_extension}")

                adm_setaf_reasoner = SimpleAdmissibleSetAfReasoner()
                adm_setaf_extensions = adm_setaf_reasoner.getModels(set_af)
                print(f"\n* Extensions Admissibles ({adm_setaf_extensions.size()}): {adm_setaf_extensions}")

                pref_setaf_reasoner = SimplePreferredSetAfReasoner()
                pref_setaf_extensions = pref_setaf_reasoner.getModels(set_af)
                print(f"\n* Extensions Préférées ({pref_setaf_extensions.size()}): {pref_setaf_extensions}")

            except jpype.JException as e_reason_java:
                print(f"❌ Erreur Java lors du raisonnement SetAF: {e_reason_java.message()}")
                # print(e_reason_java.stacktrace())
            except Exception as e_reason_py:
                print(f"❌ Erreur Python lors du raisonnement SetAF: {e_reason_py}")


        else:
             print("❌ Imports SetAF échoués. Impossible de continuer l'exemple.")

    # Gestion globale
    except ImportError as e:
        print(f"❌ Erreur d'import pour SetAF : {e}. Vérifiez le JAR 'arg.setaf'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale SetAF: {e_java.message()}")
        # print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue SetAF: {e_gen}")
        import traceback; traceback.print_exc()

### 5.6 Frameworks Étendus (Attaques sur Attaques)
<a id="5.6"></a>

Les Frameworks d'Argumentation Étendus [Modgil, 2009] permettent de modéliser des scénarios où une attaque peut elle-même être attaquée par un argument. Cela permet de représenter des concepts comme la préférence ou la priorité entre attaques de manière plus directe qu'avec les AAF standards. Tweety supporte deux variantes principales :

1.  **EAF (Extended Argumentation Frameworks)** (`org.tweetyproject.arg.extended.syntax.ExtendedTheory`): Permet aux arguments d'attaquer des attaques. Une attaque $(a, b)$ est représentée comme un objet qui peut être la cible d'une autre attaque $(c, (a, b))$.
2.  **REAF (Recursive Extended Argumentation Frameworks)** (`org.tweetyproject.arg.extended.syntax.RecursiveExtendedTheory`): Généralise les EAF en permettant aux arguments d'attaquer des attaques sur des attaques (récursivement). Une attaque étendue $(a, \alpha)$ peut avoir comme cible $\alpha$ soit un argument, soit une autre attaque étendue.

Le raisonnement sur ces frameworks étend les sémantiques classiques pour prendre en compte la réintégration potentielle d'arguments dont les attaquants sont eux-mêmes attaqués avec succès.

* **Tweety :**
    * `ExtendedTheory` / `RecursiveExtendedTheory`
    * `ExtendedAttack` (pour REAF, représente une attaque dont la cible peut être une autre attaque)
    * Raisonneurs (`org.tweetyproject.arg.extended.reasoner.*`) : `SimpleExtendedCompleteReasoner`, `SimpleRecursiveExtendedCompleteReasoner`.

In [ ]:
# --- 5.6 Frameworks Étendus (EAF / REAF) ---
print("\n--- 5.6 Frameworks Étendus (EAF / REAF) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple EAF/REAF...")
    eaf_reaf_imports_ok = False
    try:
        # Imports (inchangés)
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import Argument, Attack
        from org.tweetyproject.arg.extended.syntax import ExtendedTheory, RecursiveExtendedTheory, ExtendedAttack
        from org.tweetyproject.arg.extended.reasoner import SimpleExtendedCompleteReasoner, SimpleRecursiveExtendedCompleteReasoner
        # Ajouter l'import pour JObject si nécessaire (dépend si utilisé dans d'autres cellules)
        # from jpype import JObject

        print("✔️ Imports EAF/REAF réussis.")
        eaf_reaf_imports_ok = True

        # --- Exemple EAF (ExtendedTheory) ---
        if eaf_reaf_imports_ok:
            print("\n--- Exemple Extended AF (EAF) ---")
            eaf_theory = ExtendedTheory()
            # Arguments (inchangés)
            a_eaf = Argument("a_e"); b_eaf = Argument("b_e"); c_eaf = Argument("c_e")
            d_eaf = Argument("d_e"); e_eaf = Argument("e_e")
            eaf_theory.add(a_eaf); eaf_theory.add(b_eaf); eaf_theory.add(c_eaf)
            eaf_theory.add(d_eaf); eaf_theory.add(e_eaf)

            # *** CORRECTION: Appel de addAttack avec source et cible ***

            # 1. Ajouter les attaques standard (Arg -> Arg)
            print("   Ajout attaques standard EAF...")
            eaf_theory.addAttack(a_eaf, b_eaf)
            eaf_theory.addAttack(b_eaf, a_eaf)
            eaf_theory.addAttack(c_eaf, d_eaf)
            eaf_theory.addAttack(d_eaf, c_eaf)
            print("   ✔️ Attaques standard ajoutées.")

            # 2. Créer les objets Attack qui seront les cibles des attaques étendues
            print("   Création des objets Attack cibles...")
            # Il FAUT créer l'objet Attack AVANT de pouvoir le cibler
            attack_ba_target = Attack(b_eaf, a_eaf)
            attack_ab_target = Attack(a_eaf, b_eaf)
            attack_cd_target = Attack(c_eaf, d_eaf)
            print("   ✔️ Objets Attack cibles créés.")

            # 3. Ajouter les attaques étendues (Arg -> Attack)
            print("   Ajout attaques étendues EAF...")
            # c attaques (b->a)
            eaf_theory.addAttack(c_eaf, attack_ba_target)
            # d attaques (a->b)
            eaf_theory.addAttack(d_eaf, attack_ab_target)
            # e attaques (c->d)
            eaf_theory.addAttack(e_eaf, attack_cd_target)
            print("   ✔️ Attaques étendues ajoutées.")


            print("\nFramework EAF créé:")
            # Utiliser prettyPrint pour une meilleure lisibilité si possible
            try: print(str(eaf_theory.prettyPrint()))
            except: print(str(eaf_theory)) # Fallback si prettyPrint échoue

            # Raisonnement EAF (Complète) - Inchangé
            print(f"\nCalcul des Extensions Complètes (EAF)...")
            try:
                eaf_reasoner = SimpleExtendedCompleteReasoner()
                eaf_extensions = eaf_reasoner.getModels(eaf_theory)
                print(f"Extensions Complètes (EAF): ({eaf_extensions.size()})")
                if eaf_extensions.isEmpty(): print("  - {}")
                else:
                    for ext in eaf_extensions: print(f"  - {ext}") # Affichage standard
            except Exception as e_eaf_reason:
                print(f"❌ Erreur raisonnement EAF: {e_eaf_reason}")


            # --- Exemple REAF (RecursiveExtendedTheory) ---
            print("\n\n--- Exemple Recursive Extended AF (REAF) ---")
            reaf_theory = RecursiveExtendedTheory()
            # Arguments (inchangés)
            a_reaf = Argument("a_r"); b_reaf = Argument("b_r"); c_reaf = Argument("c_r")
            d_reaf = Argument("d_r"); e_reaf = Argument("e_r"); f_reaf = Argument("f_r")
            reaf_theory.add(a_reaf); reaf_theory.add(b_reaf); reaf_theory.add(c_reaf)
            reaf_theory.add(d_reaf); reaf_theory.add(e_reaf); reaf_theory.add(f_reaf)

            # *** CORRECTION: Appels à addAttack pour REAF ***
            # 1. Ajouter les attaques standard (Arg -> Arg)
            print("   Ajout attaques standard REAF...")
            reaf_theory.addAttack(a_reaf, b_reaf)
            reaf_theory.addAttack(b_reaf, a_reaf)
            reaf_theory.addAttack(d_reaf, c_reaf)
            reaf_theory.addAttack(c_reaf, d_reaf)
            print("   ✔️ Attaques standard ajoutées.")

            # 2. Créer les objets ExtendedAttack cibles (attaques de niveau 1)
            print("   Création des objets ExtendedAttack cibles (niveau 1)...")
            # ExtendedAttack(Argument source, Object target) où target est un Argument
            attack_ab_reaf_target = ExtendedAttack(a_reaf, b_reaf)
            attack_ba_reaf_target = ExtendedAttack(b_reaf, a_reaf)
            attack_cd_reaf_target = ExtendedAttack(c_reaf, d_reaf)
            print("   ✔️ Objets ExtendedAttack cibles (niveau 1) créés.")

            # 3. Ajouter les attaques étendues de niveau 1 (Arg -> Attaque(Arg,Arg))
            print("   Ajout attaques étendues REAF (niveau 1)...")
            # addAttack(Argument source, Object target) où target est un ExtendedAttack
            # Utiliser JObject pour caster la cible en Object si nécessaire (bonne pratique)
            reaf_theory.addAttack(c_reaf, JObject(attack_ba_reaf_target, ExtendedAttack)) # c -> (b->a)
            reaf_theory.addAttack(d_reaf, JObject(attack_ab_reaf_target, ExtendedAttack)) # d -> (a->b)
            reaf_theory.addAttack(e_reaf, JObject(attack_cd_reaf_target, ExtendedAttack)) # e -> (c->d)
            print("   ✔️ Attaques étendues (niveau 1) ajoutées.")

            # 4. Créer l'objet ExtendedAttack cible de niveau 2
            print("   Création de l'objet ExtendedAttack cible (niveau 2)...")
            # C'est l'attaque e -> (c->d) qui sera ciblée
            attack_e_cd_target_lvl2 = ExtendedAttack(e_reaf, JObject(attack_cd_reaf_target, ExtendedAttack))
            print("   ✔️ Objet ExtendedAttack cible (niveau 2) créé.")

            # 5. Ajouter l'attaque étendue de niveau 2 (Arg -> Attaque(Arg, Attaque(...)))
            print("   Ajout attaque étendue REAF (niveau 2)...")
            reaf_theory.addAttack(f_reaf, JObject(attack_e_cd_target_lvl2, ExtendedAttack)) # f -> (e -> (c->d))
            print("   ✔️ Attaque étendue (niveau 2) ajoutée.")


            print("\nFramework REAF créé:")
            try: print(str(reaf_theory.prettyPrint()))
            except: print(str(reaf_theory))

            # Raisonnement REAF (Complète) - Inchangé
            print(f"\nCalcul des Extensions Complètes (REAF)...")
            try:
                reaf_reasoner = SimpleRecursiveExtendedCompleteReasoner()
                reaf_extensions = reaf_reasoner.getModels(reaf_theory)
                print(f"Extensions Complètes (REAF): ({reaf_extensions.size()})")
                if reaf_extensions.isEmpty(): print("  - {}")
                else:
                    for ext in reaf_extensions: print(f"  - {ext}")
            except Exception as e_reaf_reason:
                 print(f"❌ Erreur raisonnement REAF: {e_reaf_reason}")


        else:
             print("❌ Imports EAF/REAF échoués. Impossible de continuer l'exemple.")

    # Gestion globale (inchangée)
    except ImportError as e: print(f"❌ Erreur d'import pour EAF/REAF : {e}. Vérifiez le JAR 'arg.extended'.")
    except jpype.JException as e_java: print(f"❌ Erreur Java générale EAF/REAF: {e_java.message()}"); print(e_java.stacktrace())
    except Exception as e_gen: print(f"❌ Erreur Python inattendue EAF/REAF: {e_gen}"); import traceback; traceback.print_exc()

### 5.7 Sémantiques Basées sur le Classement (Ranking)
<a id="5.7"></a>

Au lieu de simplement déterminer si un argument est accepté ou rejeté, les sémantiques basées sur le classement (Ranking Semantics) visent à établir un **ordre** (total ou partiel) entre les arguments, du plus "fort" ou "acceptable" au moins acceptable. Cela permet une analyse plus fine de la dialectique.

De nombreuses approches existent, basées sur des principes variés :

* **Comptage** (Attaquants/Défenseurs) : Ex: Burden-Based, Discussion-Based, Counting Semantics.
* **Propagation de valeurs** : Ex: Categorizer, Propagation Semantics.
* **Jeux/Stratégies** : Ex: Strategy-Based (Matt & Toni).
* **Tuples/Matrices** : Ex: Tuples* (Cayrol & Lagasquie-Schiex).
* **Sémantiques Graduées Itératives** : Ex: Iterated Graded Defense (Grossi & Modgil).
* **Approches Sociales/Probabilistes** : Ex: SAF-Based, Probabilistic Ranking.

Tweety fournit des implémentations pour plusieurs de ces sémantiques dans le package `org.tweetyproject.arg.rankings.reasoner.*`. Le résultat est souvent un objet `Ranking<Argument>` qui représente le classement. L'utilitaire `RankingTools.roundRanking` peut être utile pour arrondir les scores numériques.

L'exemple suivant illustre l'application de plusieurs de ces raisonneurs sur différents AAFs tirés de la littérature.

In [ ]:
# --- 5.7 Sémantiques Basées sur le Classement (Ranking) ---
print("\n--- 5.7 Sémantiques Basées sur le Classement (Ranking) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution des exemples de Ranking Semantics...")
    ranking_imports_ok = False
    try:
        # Imports
        import jpype
        from jpype.types import *

        # Syntaxe Dung
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        # Raisonneurs de Classement (Importer ceux qu'on va utiliser)
        from org.tweetyproject.arg.rankings.reasoner import (
             CategorizerRankingReasoner, BurdenBasedRankingReasoner, DiscussionBasedRankingReasoner,
             TuplesRankingReasoner, StrategyBasedRankingReasoner, SAFRankingReasoner,
             CountingRankingReasoner, PropagationRankingReasoner, IteratedGradedDefenseReasoner,
             ProbabilisticRankingReasoner # Ajouter si on l'inclut ici
        )
        # Outils Ranking
        from org.tweetyproject.arg.rankings.util import RankingTools
        # Dépendances possibles (ex: pour Probabilistic)
        from org.tweetyproject.arg.dung.semantics import Semantics
        from org.tweetyproject.math.probability import Probability

        print("✔️ Imports pour Ranking Semantics réussis.")
        ranking_imports_ok = True

        # --- Définition des AAFs exemples ---
        if ranking_imports_ok:
            print("\nDéfinition des AAFs exemples...")

            # Example 1 (Bonzon et al. AAAI 2016)
            aaf_ex1 = DungTheory()
            args1 = {name: Argument(name) for name in "abcde"}
            for arg in args1.values(): aaf_ex1.add(arg)
            attacks1 = [("a","e"), ("d","a"), ("e","d"), ("c","e"), ("b","c"), ("b","a")]
            for s, t in attacks1: aaf_ex1.add(Attack(args1[s], args1[t]))
            print("   - AAF Example 1 (Bonzon) défini.")

            # Example 2 (Pu et al. CoRR 2015, Fig 1a - pour Counting)
            aaf_ex2 = DungTheory()
            args2 = {f"x{i}": Argument(f"x{i}") for i in range(1, 5)}
            for arg in args2.values(): aaf_ex2.add(arg)
            attacks2 = [("x2","x3"), ("x2","x1"), ("x3","x2"), ("x3","x3"), ("x4","x2")]
            for s, t in attacks2: aaf_ex2.add(Attack(args2[s], args2[t]))
            print("   - AAF Example 2 (Pu) défini.")

            # Example 5 (Delobelle Thesis 2017, Fig 2.4 - pour Tuples*, Propagation)
            aaf_ex5 = DungTheory()
            args5 = {name: Argument(name) for name in "abcdefghij"}
            for arg in args5.values(): aaf_ex5.add(arg)
            attacks5 = [("a","b"), ("b","c"), ("b","f"), ("d","g"), ("d","f"),
                        ("e","h"), ("e","d"), ("e","i"), ("h","g"), ("j","i")]
            for s, t in attacks5: aaf_ex5.add(Attack(args5[s], args5[t]))
            print("   - AAF Example 5 (Delobelle) défini.")

            # Example 4a (Matt & Toni JELIA 2008, Fig 2 - pour StrategyBased)
            aaf_ex4a = DungTheory()
            args4a = {name: Argument(name) for name in "abcdefg"}
            for arg in args4a.values(): aaf_ex4a.add(arg)
            attacks4a = [("b","a"),("c","a"),("d","a"),("f","a"),("e","d"),("g","f")]
            for s, t in attacks4a: aaf_ex4a.add(Attack(args4a[s], args4a[t]))
            print("   - AAF Example 4a (Matt & Toni) défini.")


            # --- Application des Raisonneurs ---
            print("\n--- Application des Raisonneurs de Classement ---")

            # 1. Categorizer
            print("\n* Categorizer:")
            try:
                r_cat = CategorizerRankingReasoner()
                rank_cat1 = r_cat.getModel(aaf_ex1)
                rank_cat2 = r_cat.getModel(aaf_ex2)
                print(f"  - AAF Ex1: {RankingTools.roundRanking(rank_cat1, 2)}")
                print(f"  - AAF Ex2: {RankingTools.roundRanking(rank_cat2, 3)}")
            except Exception as e: print(f"   ❌ Erreur Categorizer: {e}")

            # 2. Burden-Based
            print("\n* Burden-Based:")
            try:
                r_bb = BurdenBasedRankingReasoner()
                rank_bb1 = r_bb.getModel(aaf_ex1)
                rank_bb5 = r_bb.getModel(aaf_ex5)
                print(f"  - AAF Ex1: {rank_bb1}")
                print(f"  - AAF Ex5: {rank_bb5}")
            except Exception as e: print(f"   ❌ Erreur BurdenBased: {e}")

            # 3. Discussion-Based
            print("\n* Discussion-Based:")
            try:
                r_db = DiscussionBasedRankingReasoner()
                rank_db1 = r_db.getModel(aaf_ex1)
                rank_db2 = r_db.getModel(aaf_ex2) # Utiliser Ex2 comme dans Java
                print(f"  - AAF Ex1: {rank_db1}")
                print(f"  - AAF Ex2: {rank_db2}")
            except Exception as e: print(f"   ❌ Erreur DiscussionBased: {e}")

            # 4. Tuples*
            print("\n* Tuples*:")
            try:
                r_tup = TuplesRankingReasoner()
                rank_tup5 = r_tup.getModel(aaf_ex5)
                print(f"  - AAF Ex5 (Classement): {rank_tup5}")
                # prettyPrintTupledValues n'est pas directement appelable en Python
                # Mais on peut essayer d'accéder aux valeurs via une méthode si elle existe
                # ou simplement afficher le classement qui est la sortie principale.
                # print(f"  - AAF Ex5 (Valeurs): {r_tup.prettyPrintTupledValues()}") # N'existe pas
            except Exception as e: print(f"   ❌ Erreur Tuples: {e}")

            # 5. Strategy-Based (Matt & Toni)
            print("\n* Strategy-Based:")
            try:
                r_sb = StrategyBasedRankingReasoner()
                rank_sb4a = r_sb.getModel(aaf_ex4a)
                print(f"  - AAF Ex4a: {RankingTools.roundRanking(rank_sb4a, 3)}")
            except Exception as e: print(f"   ❌ Erreur StrategyBased: {e}")

            # 6. SAF-Based
            print("\n* SAF-Based (SimpleProduct):")
            try:
                r_saf = SAFRankingReasoner() # Utilise SimpleProductSemantics par défaut
                rank_saf1 = r_saf.getModel(aaf_ex1)
                print(f"  - AAF Ex1: {RankingTools.roundRanking(rank_saf1, 2)}")
            except Exception as e: print(f"   ❌ Erreur SAFBased: {e}")

            # 7. Counting Semantics
            print("\n* Counting Semantics:")
            try:
                # CountingRankingReasoner(double decay_factor, double tolerance)
                r_count = CountingRankingReasoner(0.98, 0.001)
                rank_count1 = r_count.getModel(aaf_ex1)
                rank_count2 = r_count.getModel(aaf_ex2)
                print(f"  - AAF Ex1: {RankingTools.roundRanking(rank_count1, 2)}")
                print(f"  - AAF Ex2: {RankingTools.roundRanking(rank_count2, 2)}")
            except Exception as e: print(f"   ❌ Erreur Counting: {e}")

            # 8. Propagation Semantics (une variante)
            print("\n* Propagation Semantics (Variant 1 - Set-based):")
            try:
                # PropagationRankingReasoner(double epsilon, boolean multiset, PropagationVariant variant)
                # Variant 1 = PROPAGATION1
                r_prop1 = PropagationRankingReasoner(0.75, False, PropagationRankingReasoner.PropagationSemantics.PROPAGATION1)
                rank_prop1_ex5 = r_prop1.getModel(aaf_ex5)
                print(f"  - AAF Ex5 (ε=0.75, Set): {rank_prop1_ex5}")
            except Exception as e: print(f"   ❌ Erreur Propagation: {e}")

            # 9. Iterated Graded Defense (Optionnel, peut être complexe/lent)
            # print("\n* Iterated Graded Defense:")
            # try:
            #     r_igd = IteratedGradedDefenseReasoner()
            #     # getAllMNCompleteExtensions(DungTheory, int m, int n)
            #     igd_exts = r_igd.getAllMNCompleteExtensions(aaf_ex1, 2, 2) # Exemple m=2, n=2
            #     print(f"  - AAF Ex1 (m=2, n=2): {igd_exts}")
            # except Exception as e: print(f"   ❌ Erreur IGD: {e}")

            # 10. Probabilistic Ranking (Optionnel)
            # print("\n* Probabilistic Ranking (Grounded, p=0.5):")
            # try:
            #     r_prob = ProbabilisticRankingReasoner(Semantics.GROUNDED_SEMANTICS, Probability(0.5), True)
            #     rank_prob2 = r_prob.getModel(aaf_ex2)
            #     print(f"  - AAF Ex2: {rank_prob2}")
            # except Exception as e: print(f"   ❌ Erreur Probabilistic Ranking: {e}")

        else:
             print("❌ Imports pour Ranking Semantics échoués. Impossible de continuer.")

    # Gestion globale
    except ImportError as e:
        print(f"❌ Erreur d'import pour Ranking Semantics : {e}. Vérifiez les JARs 'arg.rankings', 'arg.dung', 'math'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale Ranking: {e_java.message()}")
        print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue Ranking: {e_gen}")
        import traceback; traceback.print_exc()

### 5.8 Argumentation Probabiliste
<a id="5.8"></a>

L'argumentation probabiliste introduit la notion d'incertitude dans les cadres d'argumentation. Cette incertitude peut porter sur :

1.  **L'existence des arguments ou des attaques** : Le graphe lui-même est incertain.
2.  **L'acceptabilité des arguments** : On peut avoir des degrés de croyance sur le fait qu'un argument soit accepté.

Tweety se concentre principalement sur le premier type d'incertitude via les **Distributions de Probabilité sur les Sous-Graphes**.

* **`SubgraphProbabilityFunction`**: Représente une distribution de probabilité sur l'ensemble des sous-graphes possibles d'un AAF de référence. Chaque sous-graphe (un AAF potentiel) a une probabilité associée. Souvent initialisée de manière uniforme.
* **Acceptabilité Probabiliste (`getAcceptanceProbability`)**: Étant donné une `SubgraphProbabilityFunction` et une sémantique de Dung, on peut calculer la probabilité qu'un argument (ou un ensemble d'arguments) soit accepté. Cela correspond à la somme des probabilités des sous-graphes où cet argument (ou ensemble) est accepté selon la sémantique choisie.
* **Divisions (`Division`)**: Une partition des arguments en trois ensembles : acceptés (IN), rejetés (OUT), indécis (UNDEC). Utilisé pour raisonner sur des résultats d'évaluation plus fins.
* **Loteries d'Argumentation (`ArgumentationLottery`, `UtilityFunction`)**: Permettent de modéliser des scénarios de décision sous incertitude, en associant des utilités aux différentes issues possibles (divisions) et en calculant l'utilité attendue en fonction des probabilités d'acceptation.

L'exemple suivant s'inspire de `LotteryExample.java` pour illustrer ces concepts.

In [ ]:
# --- 5.8 Argumentation Probabiliste ---
print("\n--- 5.8 Argumentation Probabiliste ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple d'argumentation probabiliste...")
    prob_imports_ok = False
    try:
        # Imports (inchangés)
        import jpype
        from jpype.types import *
        from java.util import Collection, List as JavaList, Set as JavaSet

        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.reasoner import AbstractExtensionReasoner, SimpleGroundedReasoner
        from org.tweetyproject.arg.dung.semantics import Extension, Semantics
        from org.tweetyproject.arg.dung.divisions import Division

        from org.tweetyproject.arg.prob.lotteries import SubgraphProbabilityFunction, ArgumentationLottery, UtilityFunction
        from org.tweetyproject.math.probability import Probability

        print("✔️ Imports pour Argumentation Probabiliste réussis.")
        prob_imports_ok = True

        # --- Création de l'AAF de base ---
        if prob_imports_ok:
            print("\n1. Création de l'AAF de base...")
            theory_prob = DungTheory()
            a = Argument("a"); b = Argument("b"); c = Argument("c")
            theory_prob.add(a); theory_prob.add(b); theory_prob.add(c)
            theory_prob.add(Attack(a, b)); theory_prob.add(Attack(b, a)); theory_prob.add(Attack(c, b))
            print("   AAF: ", theory_prob)

            # --- Raisonnement standard (inchangé) ---
            print("\n2. Extensions Grounded (pour référence)...")
            reasoner_gr = SimpleGroundedReasoner()
            extensions_gr_coll = reasoner_gr.getModels(theory_prob)
            print("   Extensions Grounded:", extensions_gr_coll)

            # --- Fonction de Probabilité sur Sous-Graphes (inchangé) ---
            print("\n3. Création d'une fonction de probabilité uniforme sur les sous-graphes...")
            prob_function = SubgraphProbabilityFunction(theory_prob)
            print(f"   Nombre de sous-graphes possibles: {prob_function.size()}")

            # --- Calcul d'Acceptabilité Probabiliste ---
            print("\n4. Calcul des probabilités d'acceptation (Sémantique Grounded)...")
            target_semantics = Semantics.GROUNDED_SEMANTICS
            try:
                prob_obj_a = prob_function.getAcceptanceProbability(a, target_semantics)
                prob_obj_b = prob_function.getAcceptanceProbability(b, target_semantics)
                prob_obj_c = prob_function.getAcceptanceProbability(c, target_semantics)
                val_a = prob_obj_a.doubleValue(); val_b = prob_obj_b.doubleValue(); val_c = prob_obj_c.doubleValue()
                print(f"   P(Accepté({a})) = {val_a:.4f}")
                print(f"   P(Accepté({b})) = {val_b:.4f}")
                print(f"   P(Accepté({c})) = {val_c:.4f}")

                print("\n   Calcul probabilité pour Division ({a, c}, {b})...")
                in_set = Extension(); in_set.add(a); in_set.add(c)
                out_set = Extension(); out_set.add(b)
                # *** CORRECTION CONSTRUCTEUR Division : seulement IN et OUT ***
                example_division = Division(in_set, out_set)
                prob_obj_div = prob_function.getAcceptanceProbability(example_division, target_semantics)
                val_div = prob_obj_div.doubleValue()
                print(f"   P(Division={example_division}) = {val_div:.4f}") # Devrait fonctionner maintenant

            except jpype.JException as e_prob_calc_java:
                 if "'doubleValue'" in str(e_prob_calc_java.message()): print("   ⚠️ Erreur: La méthode .doubleValue() n'existe pas sur l'objet Probability.")
                 else: print(f"   ❌ Erreur Java calcul probabilités: {e_prob_calc_java.message()}")
            except AttributeError as e_attr:
                 if "'doubleValue'" in str(e_attr): print("   ⚠️ Erreur: La méthode .doubleValue() n'existe pas sur l'objet Probability.")
                 else: print(f"   ❌ Erreur Python calcul probabilités (Attr): {e_attr}")
            except Exception as e_prob_calc: print(f"   ❌ Erreur calcul probabilités: {e_prob_calc}")


            # --- Loteries et Utilités ---
            print("\n5. Loteries d'Argumentation et Utilité Attendue...")
            try:
                # *** CORRECTION APPEL : Retirer le second argument (sémantique) ***
                standard_divisions_coll = Division.getStandardDivisions(theory_prob)
                # *** CORRECTION AFFICHAGE : Utiliser str() pour la sémantique ***
                print(f"   Divisions standard (basées sur {str(target_semantics)} extensions): {standard_divisions_coll}")

                if not standard_divisions_coll.isEmpty():
                    lottery = ArgumentationLottery(standard_divisions_coll, prob_function, target_semantics)
                    print("\n   Loterie basée sur les divisions standard:")
                    print(f"     {lottery}")

                    utility_func = UtilityFunction()
                    from java.util import ArrayList # Assurer l'import ici
                    standard_divisions_list = ArrayList(standard_divisions_coll)

                    # Assigner des utilités (inchangé, mais dépend de standard_divisions_list correct)
                    Division_class = jpype.JClass("org.tweetyproject.arg.dung.divisions.Division") # Pour cast JObject
                    if standard_divisions_list.size() >= 1: utility_func.put(JObject(standard_divisions_list.get(0), Division_class), 10.0)
                    if standard_divisions_list.size() >= 2: utility_func.put(JObject(standard_divisions_list.get(1), Division_class), -5.0)

                    if not utility_func.isEmpty():
                        print("\n   Fonction d'utilité exemple:")
                        print(f"     {utility_func}")
                        expected_utility = utility_func.getExpectedUtility(lottery)
                        # Utiliser doubleValue() ici aussi pour le formatage
                        print(f"\n   Utilité Attendue de la loterie: {expected_utility.doubleValue():.4f}")
                    else: print("\n   Pas assez de divisions standard ou erreur pour définir une fonction d'utilité exemple.")
                else: print("   Aucune division standard trouvée pour créer la loterie.")

            except jpype.JException as e_lottery_java: print(f"   ❌ Erreur Java (Loteries/Utilité): {e_lottery_java.message()}")
            except Exception as e_lottery_py: print(f"   ❌ Erreur Python (Loteries/Utilité): {e_lottery_py}"); import traceback; traceback.print_exc()

        else: print("❌ Imports pour Argumentation Probabiliste échoués. Impossible de continuer.")

    # Gestion globale (inchangée)
    except ImportError as e: print(f"❌ Erreur d'import Prob: {e}.")
    except jpype.JException as e_java: print(f"❌ Erreur Java générale Prob: {e_java.message()}"); print(e_java.stacktrace())
    except Exception as e_gen: print(f"❌ Erreur Python inattendue Prob: {e_gen}"); import traceback; traceback.print_exc()

---

## Navigation

- **Precedent**: [Tweety-6-Structured-Argumentation.ipynb](Tweety-6-Structured-Argumentation.ipynb)
- **Suivant**: [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb)
- **Index**: [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb)
